In [314]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import pennylane as qml
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

In [315]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"Using: {torch.cuda.get_device_name(0)}")
    print(f"CUDA: {torch.version.cuda}")
else:
    print("CUDA is not available. Using CPU.")

Using: NVIDIA GeForce RTX 4070 Ti SUPER
CUDA: 12.4


In [316]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pennylane as qml
import numpy as np

# Função para criar o circuito quântico
def quanvolution_circuit(inputs, weights):
    # Inicializar o estado de entrada com a imagem (entradas normais)
    qml.templates.AngleEmbedding(inputs, wires=range(4), rotation='X')  # Ajuste para 4 qubits
    
    # Operações de convolução quântica (Camadas de Hadamard, RZ, etc.)
    for i in range(4):  # Ajuste para 4 qubits
        qml.RZ(weights[i], wires=i)
        qml.Hadamard(wires=i)
    
    # Retornar o valor esperado no qubit 0
    return qml.expval(qml.PauliZ(0))

# Definir a camada de convolução quântica
class QuanvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(QuanvolutionLayer, self).__init__()
        self.weights = nn.Parameter(torch.randn(4, dtype=torch.float64))  # 4 qubits
        
    def forward(self, x):
        # Preparar a entrada da imagem para o circuito quântico
        x = x.view(-1)  # Flatten
        
        # Criar o QNode para simular o circuito quântico
        qnode = qml.QNode(quanvolution_circuit, qml.device('default.qubit', wires=4))
        
        # Obter o valor esperado diretamente do QNode
        output = qnode(x, self.weights)
        return output

# Função para aplicar o circuito quântico em regiões 2x2
def apply_quantum_convolution(image, quantum_layer):
    """Aplica o circuito quântico em blocos 2x2 da imagem."""
    out = np.zeros((image.shape[0] // 2, image.shape[1] // 2, 4))  # Ajustar para saída de tamanho reduzido

    # Loop sobre as coordenadas do canto superior esquerdo de quadrados 2x2
    for j in range(0, image.shape[0] - 1, 2):  # Percorrendo a altura da imagem (garante que não exceda os limites)
        for k in range(0, image.shape[1] - 1, 2):  # Percorrendo a largura da imagem (garante que não exceda os limites)
            # Processar uma região 2x2 da imagem com o circuito quântico
            q_results = quantum_layer(
                torch.tensor([
                    image[j, k],
                    image[j, k + 1],
                    image[j + 1, k],
                    image[j + 1, k + 1]
                ], dtype=torch.float64).to("cuda")  # Mover para a GPU, se necessário
            )  # Obter o valor esperado diretamente do QNode
            
            # Atribuir os valores de expectativa aos diferentes canais do pixel de saída
            out[j // 2, k // 2] = q_results.detach().cpu().numpy()  # .detach() para remover o rastreamento de gradientes e .cpu() para mover para a CPU
    
    return out


# Definir a Rede Neural com a camada quântica
class QuanvolutionNet(nn.Module):
    def __init__(self):
        super(QuanvolutionNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)  # Convolução
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # MaxPooling para reduzir a dimensionalidade
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # Segunda camada convolucional
        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # Camada totalmente conectada após a convolução
        self.fc2 = nn.Linear(128, 2)  # Para 2 classes (0 ou 1)
        self.quantum_layer = QuanvolutionLayer(128, 4)  # Camada quântica após a redução

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Passar pela primeira camada convolucional + Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Passar pela segunda camada convolucional + Pooling
        x = x.view(-1, 32 * 8 * 8)  # Flatten para a camada totalmente conectada
        x = torch.relu(self.fc1(x))  # Passar pela camada totalmente conectada
        x = self.fc2(x)  # Passar pela camada de saída
        
        # Aplicando a convolução quântica
        x = apply_quantum_convolution(x, self.quantum_layer)  # Aqui aplicamos o circuito quântico
        
        return x

# Exemplo de uso
model = QuanvolutionNet().to(device)  # Certifique-se de que o modelo está na GPU
print(model)


QuanvolutionNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (quantum_layer): QuanvolutionLayer()
)


In [317]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convertendo para escala de cinza
    transforms.ToTensor(),
])

In [318]:
# Carregando os conjuntos de dados de treino e teste a partir das pastas
train_dataset = datasets.ImageFolder(root="/home/eflammere/QuantumIC/Datasets/INbreast/png/train", transform=transform)
test_dataset = datasets.ImageFolder(root="/home/eflammere/QuantumIC/Datasets/INbreast/png/test", transform=transform)

# Criando DataLoaders para carregar os dados em batches
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Instanciando o modelo
model = QuanvolutionNet().to(device)

# Definindo o otimizador e a função de perda
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [319]:
# Treinando o modelo
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        # Garantir que os inputs e labels sejam tensores no dispositivo correto (CPU ou GPU)
        inputs, labels = inputs.to(device), labels.to(device)

        print(f"inputs.shape: {inputs.shape}, labels.shape: {labels.shape}")

        # Zerar os gradientes acumulados
        optimizer.zero_grad()

        # Passando as imagens pela rede
        outputs = model(inputs)

        # Convertendo outputs e labels para o tipo de tensor correto
        outputs = torch.tensor(outputs, dtype=torch.float32).to(inputs.device)  # Mover para o dispositivo correto, se necessário
        labels = torch.tensor(labels, dtype=torch.long).to(inputs.device)  # Certifique-se de que os rótulos sejam longos (inteiros)

        # Calculando a perda
        loss = criterion(outputs, labels)
        loss.backward()

        # Atualizando os parâmetros
        optimizer.step()

        # Atualizando a perda total
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


inputs.shape: torch.Size([128, 1, 64, 64]), labels.shape: torch.Size([128])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x2048 and 8192x128)

In [ ]:
# Avaliando o modelo
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Passando as imagens pela rede
        outputs = model(inputs)
        
        # Calculando as predições
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy}%")